In [15]:
import numpy as np

import torch
import time

from falkon import LogisticFalkon
from falkon.kernels import GaussianKernel
from falkon.options import FalkonOptions
from falkon.gsc_losses import WeightedCrossEntropyLoss

from scipy.spatial.distance import pdist


In [16]:
def get_logflk_config(M,flk_sigma,lam,weight,iter=[1000000],seed=None,cpu=False):
    # it returns logfalkon parameters
    return {
            'kernel' : GaussianKernel(sigma=flk_sigma),
            'M' : M, #number of Nystrom centers,
            'penalty_list' : [lam], # list of regularization parameters,
            'iter_list' : [iter], #list of number of CG iterations,
            'options' : FalkonOptions(cg_tolerance=np.sqrt(1e-7), keops_active='no', use_cpu=cpu, debug = False),
            'seed' : seed, # (int or None), the model seed (used for Nystrom center selection) is manually set,
            'loss' : WeightedCrossEntropyLoss(kernel=GaussianKernel(sigma=flk_sigma), neg_weight=weight),
            }

def trainer(X,Y,flk_config):
    # trainer for logfalkon model
    Xtorch=torch.from_numpy(X)
    Ytorch=torch.from_numpy(Y)
    model = LogisticFalkon(**flk_config)
    model.fit(Xtorch, Ytorch)
    return model.predict(Xtorch).numpy()

def compute_t(preds,Y,weight):
    # it returns extended log likelihood ratio from predictions
    diff = weight*np.sum(1 - np.exp(preds[Y==0]))
    return 2 * (diff + np.sum(preds[Y==1]))


def candidate_sigma(data, perc=90):
    # this function gives an estimate of the width of the gaussian kernel
    # use on a (small) sample of reference data (standardize first if necessary)
    pairw = pdist(data)
    return round(np.percentile(pairw,perc),1)

In [17]:
N_0 = 20000 # size of reference sample
N_1 = 2000 # size of the data sample
N = N_0 + N_1
weight = N_1/N_0

M = 1000
lam = 1e-6
# sigma is set later

In [18]:
# generate data

rng = np.random.default_rng(seed=0)

scale = 1.2 # if scale = 0 we are in the null hypothesis 
dim = 4

# estimate the gaussian width on a reference sample (not too large, the pairwise distance is expensive to compute)

ref_sample_for_sigma = rng.multivariate_normal(mean=dim*[1],cov=np.diag(np.full(dim,1)),size=1000)

flk_sigma = candidate_sigma(ref_sample_for_sigma, perc=90)
print(f"Estimated Gaussian width: {flk_sigma}")


X = np.zeros(shape=(N,dim))

X[:N_0,:] = rng.multivariate_normal(mean=dim*[1],cov=np.diag(np.full(dim,1)),size=N_0) # reference
X[N_0:,:] = rng.multivariate_normal(mean=dim*[scale],cov=np.diag(np.full(dim,1)),size=N_1) # data


# initialize labes
Y = np.zeros(shape=(N,1))
Y[N_0:,:] = np.ones((N_1,1)) # flip data labels to one for data


Estimated Gaussian width: 3.9


In [19]:
# train and evaluate
flk_config = get_logflk_config(M,flk_sigma,lam,weight=weight,iter=1000000,seed=None,cpu=False)
flk_config['seed'] = 0 # seed for center selection

st_time = time.time()
preds = trainer(X,Y,flk_config)

t = compute_t(preds,Y,weight)
dt = round(time.time()-st_time,2)

print(f"Value of the test statistic: {t}\nTraining and evaluation time: {dt}")

Iteration 0 - penalty 1.000000e-06 - sub-iterations 1000000
Value of the test statistic: 364.617466781465
Training and evaluation time: 0.38
